In [1]:
import os
import glob
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [2]:
video_files = glob.glob('video_footages/*')
video_files

['video_footages\\Raw Video_ Pittsburg Neighborhood Drive-By Shootings.mp4',
 'video_footages\\Shooting captured by surveillance camera in Parma.mp4',
 'video_footages\\Shopping, People, Commerce, Mall, Many, Crowd, Walking   Free Stock video footage   YouTube.mp4',
 'video_footages\\snow dispute.mp4',
 'video_footages\\Surveillance camera captures tanks entering Ukraine via Belarus border.mp4',
 'video_footages\\trail cam video creatures of the night.mp4',
 'video_footages\\Unbelievably busy bicycle crossing in Amsterdam.mp4',
 'video_footages\\vecteezy_people-crossing-the-road-on-zebra-tallin_28257759.mp4',
 'video_footages\\차흐르듯 (1).mp4',
 'video_footages\\차흐르듯.mp4']

In [4]:
video_num = 1
sub_type = 'MOG2'

backSubMOG = cv2.createBackgroundSubtractorMOG2(varThreshold=16, detectShadows=False)

#backSubKNN = cv2.createBackgroundSubtractorKNN(dist2Threshold=1000, detectShadows=False)

area_thresh = 500
new_h =  300 #int(height / 1.5)
new_w =  600 #int(width / 1.5)
count = 0

cap = cv2.VideoCapture(video_files[video_num])

while(cap.isOpened()):
    
    ret, frame = cap.read()
    if ret == True:
        count += 1
        frame = cv2.resize(frame, (new_w, new_h))
        #blur = cv2.medianBlur(frame, 11)
        blur = cv2.GaussianBlur(frame, (11, 11), 0)
        fgMaskMOG = backSubMOG.apply(blur)
        #fgMaskKNN = backSubKNN.apply(blur)

        contours, _ = cv2.findContours(fgMaskMOG, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_TC89_L1)
        bbox_frame = frame.copy()
        for cnt in contours:
            x,y,w,h = cv2.boundingRect(cnt)
            area = w*h
            if area > area_thresh:
                cv2.rectangle(bbox_frame, (x, y), (x+w, y+h), (0, 255, 0), thickness = 2)

        cv2.imshow('Original Video', frame)
        cv2.imshow('Fg_mask_MOG', fgMaskMOG)
        #cv2.imshow('Fg_mask_KNN', fgMaskKNN)
        cv2.imshow('Bounding Boxes', bbox_frame)

        pressedKey = cv2.waitKey(30) & 0xFF
        if pressedKey == ord('q'):
            break
        elif pressedKey == ord('s'):
            cv2.imwrite(f'saved images for essay/MOG vs KNN/{video_num}_{count}_Original_Frame.jpg',frame)
            cv2.imwrite(f'saved images for essay/MOG vs KNN/{video_num}_{count}_Fg_mask_MOG.jpg', fgMaskMOG)
            cv2.imwrite(f'saved images for essay/MOG vs KNN/{video_num}_{count}_Bounding_Boxes.jpg', bbox_frame)

    else:
        break

cap.release()

cv2.destroyAllWindows()